In [1]:
# Only for Google Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd
import math
import statistics
import numpy as np

In [3]:
"""
# for local run

# data folder ubication
data_folder = os.getcwd() + '\BTCUSDT_1hs.csv'
print(data_folder)

# load data from binance
data = pd.read_csv(data_folder)
"""

"\n# for local run\n\n# data folder ubication\ndata_folder = os.getcwd() + '\\BTCUSDT_1hs.csv'\nprint(data_folder)\n\n# load data from binance\ndata = pd.read_csv(data_folder)\n"

In [4]:
# for run in Google Colab

data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data Science, Artificial Intelligence, Machine Learning (Python)/Python/LSTM_BTC/BTCUSDT_1hs.csv")

In [5]:
data

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,1.621987e+12,2021-05-26 00:00:00,BTC/USDT,38324.72,38386.53,38282.78,38288.31,173.417801,6.650289e+06,3765.0
1,1.621984e+12,2021-05-25 23:00:00,BTC/USDT,38402.99,38467.22,37946.55,38324.72,2194.475099,8.392752e+07,60933.0
2,1.621980e+12,2021-05-25 22:00:00,BTC/USDT,37937.81,38753.98,37906.77,38405.00,3692.719609,1.412773e+08,98646.0
3,1.621976e+12,2021-05-25 21:00:00,BTC/USDT,37603.80,38110.00,37420.40,37937.79,2369.798232,8.940948e+07,69367.0
4,1.621973e+12,2021-05-25 20:00:00,BTC/USDT,37292.69,37794.72,37250.01,37598.77,2513.621469,9.445717e+07,62137.0
...,...,...,...,...,...,...,...,...,...,...
35697,1.502957e+09,2017-08-17 08-AM,BTC/USDT,4349.99,4377.85,4333.32,4360.69,0.949900,4.139700e+03,NaN
35698,1.502953e+09,2017-08-17 07-AM,BTC/USDT,4324.35,4349.99,4287.41,4349.99,4.440000,1.924106e+04,NaN
35699,1.502950e+09,2017-08-17 06-AM,BTC/USDT,4315.32,4345.45,4309.37,4324.35,7.230000,3.128231e+04,NaN
35700,1.502946e+09,2017-08-17 05-AM,BTC/USDT,4308.83,4328.69,4291.37,4315.32,23.230000,1.003048e+05,NaN


In [6]:
data = data.drop(['tradecount', 'unix'], axis=1)
data = data.reset_index()
data = data.sort_values('index', ascending=False)
data = data.drop(['index'], axis=1)
data.reset_index(drop=True, inplace=True)
data = data.reset_index()
filas = len(data)
list(data.columns.values)

data['predict'] = data['close'] > data['open']

In [7]:
data

,index,date,symbol,open,high,low,close,Volume BTC,Volume USDT,predict
0,0,2017-08-17 04-AM,BTC/USDT,16199.91,16199.91,4261.32,4308.83,44.510000,1.909529e+05,False
1,1,2017-08-17 05-AM,BTC/USDT,4308.83,4328.69,4291.37,4315.32,23.230000,1.003048e+05,True
2,2,2017-08-17 06-AM,BTC/USDT,4315.32,4345.45,4309.37,4324.35,7.230000,3.128231e+04,True
3,3,2017-08-17 07-AM,BTC/USDT,4324.35,4349.99,4287.41,4349.99,4.440000,1.924106e+04,True
4,4,2017-08-17 08-AM,BTC/USDT,4349.99,4377.85,4333.32,4360.69,0.949900,4.139700e+03,True
...,...,...,...,...,...,...,...,...,...,...
35697,35697,2021-05-25 20:00:00,BTC/USDT,37292.69,37794.72,37250.01,37598.77,2513.621469,9.445717e+07,True
35698,35698,2021-05-25 21:00:00,BTC/USDT,37603.80,38110.00,37420.40,37937.79,2369.798232,8.940948e+07,True
35699,35699,2021-05-25 22:00:00,BTC/USDT,37937.81,38753.98,37906.77,38405.00,3692.719609,1.412773e+08,True
35700,35700,2021-05-25 23:00:00,BTC/USDT,38402.99,38467.22,37946.55,38324.72,2194.475099,8.392752e+07,False


In [8]:
# create WaveTrend Oscillator [LazyBear]
n1 = 10  # "Channel Length"
n2 = 21  # "Average Length"
obLevel1 = 60  # "Over Bought Level 1"
obLevel2 = 53  # "Over Bought Level 2"
osLevel1 = 60  # "Over Sold Level 1"
osLevel2 = 53  # "Over Sold Level 2"

data['ap'] = (data['high'] + data['low'] + data['close']) / 3
data['esa'] = data['ap'].ewm(span=n1).mean()
data['d'] = abs(data['ap'] - data['esa']).ewm(span=n1).mean()
data['ci'] = (data['ap'] - data['esa']) / (0.015 * data['d'])

data['wavetrend_1'] = data['ci'].ewm(span=n2).mean()
data['wavetrend_2'] = data['wavetrend_1'].rolling(window=4).mean()
data['wavetrend_delta'] = data['wavetrend_1'] - data['wavetrend_2']

In [9]:
data

,index,date,symbol,open,high,low,close,Volume BTC,Volume USDT,predict,ap,esa,d,ci,wavetrend_1,wavetrend_2,wavetrend_delta
0,0,2017-08-17 04-AM,BTC/USDT,16199.91,16199.91,4261.32,4308.83,44.510000,1.909529e+05,False,8256.686667,8256.686667,0.000000,NaN,NaN,NaN,NaN
1,1,2017-08-17 05-AM,BTC/USDT,4308.83,4328.69,4291.37,4315.32,23.230000,1.003048e+05,True,4311.793333,6086.995333,976.361100,-121.212121,-121.212121,NaN,NaN
2,2,2017-08-17 06-AM,BTC/USDT,4315.32,4345.45,4309.37,4324.35,7.230000,3.128231e+04,True,4326.390000,5379.243688,1007.110612,-69.694674,-94.226792,NaN,NaN
3,3,2017-08-17 07-AM,BTC/USDT,4324.35,4349.99,4287.41,4349.99,4.440000,1.924106e+04,True,4329.130000,5033.278015,907.297935,-51.739566,-78.695208,NaN,NaN
4,4,2017-08-17 08-AM,BTC/USDT,4349.99,4377.85,4333.32,4360.69,0.949900,4.139700e+03,True,4357.286667,4839.219284,785.187121,-40.918706,-67.861223,-90.498836,22.637613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35697,35697,2021-05-25 20:00:00,BTC/USDT,37292.69,37794.72,37250.01,37598.77,2513.621469,9.445717e+07,True,37547.833333,37712.280490,261.797694,-41.876396,-11.077599,-6.391099,-4.686500
35698,35698,2021-05-25 21:00:00,BTC/USDT,37603.80,38110.00,37420.40,37937.79,2369.798232,8.940948e+07,True,37822.730000,37732.362219,230.628619,26.122165,-7.695802,-7.014090,-0.681713
35699,35699,2021-05-25 22:00:00,BTC/USDT,37937.81,38753.98,37906.77,38405.00,3692.719609,1.412773e+08,True,38355.250000,37845.614543,281.357135,120.756480,3.981678,-5.697361,9.679039
35700,35700,2021-05-25 23:00:00,BTC/USDT,38402.99,38467.22,37946.55,38324.72,2194.475099,8.392752e+07,False,38246.163333,37918.441596,289.787063,75.393690,10.473679,-1.079511,11.553190


In [10]:
# Create Squeeze Momentum Indicator [LazyBear]
length = 20  # "BB Length
length_div2 = int(length / 2)
mult = 2.0  # "BB MultFactor"
length2 = 20  # title="HMA Length"
source = open  # "Source"
lengthKC = 20  # "KC Length"
multKC = 1.5  # "KC MultFactor"
useTrueRange = True  # "Use TrueRange (KC)"

data['sqzwma1'] = 0
data['sqzwma2'] = 0
data['hullma'] = 0
data['dev_std'] = 0
data['TrueRange'] = 0
data['rangewma1'] = 0
data['rangewma2'] = 0
data['rangema'] = 0

f = -1
for x in range(filas):
    f = f + 1
    if f > 0:
        data['TrueRange'][f] = max((data['high'][f] - data['low'][f]), (abs(data['high'][f] - data['close'][f - 1])),
                                   (abs(data['low'][f] - data['close'][f - 1])))

wei = 0
for x in range(length + 1):
    wei = wei + x

wei_div2 = 0
for x in range(length_div2 + 1):
    wei_div2 = wei_div2 + x

f = -1
for x in range(filas):
    f = f + 1
    if f > length - 2:
        sum = 0
        sumtr = 0
        i = length
        ff = f
        for y in range(length):
            sum = sum + i * data['open'][ff]
            sumtr = sumtr + i * data['TrueRange'][ff]
            ff = ff - 1
            i = i - 1

        sum_div2 = 0
        sumtr2 = 0
        i_div2 = length_div2
        ff = f
        for y in range(length_div2):
            sum_div2 = sum_div2 + i_div2 * data['open'][ff]
            sumtr2 = sumtr2 + i_div2 * data['TrueRange'][ff]
            ff = ff - 1
            i_div2 = i_div2 - 1

        data['sqzwma1'][f] = sum / wei
        data['sqzwma2'][f] = sum_div2 / wei_div2
        data['rangewma1'][f] = sumtr / wei
        data['rangewma2'][f] = sumtr2 / wei_div2

lengthsqrt = round(math.sqrt(length))
f = -1

weisqrt = 0
for x in range(lengthsqrt + 1):
    weisqrt = weisqrt + x

for x in range(filas):
    f = f + 1
    if f > length + lengthsqrt - 2:
        sum = 0
        sumtr = 0
        i = lengthsqrt
        ff = f
        for y in range(lengthsqrt):
            sum = sum + i * (2 * data['sqzwma1'][ff] - data['sqzwma2'][ff])
            sumtr = sumtr + i * (2 * data['rangewma1'][ff] - data['rangewma2'][ff])
            ff = ff - 1
            i = i - 1

        data['hullma'][f] = sum / weisqrt
        data['rangema'][f] = sumtr / weisqrt

f = -1
for x in range(filas):
    f = f + 1
    if f > length - 2:
        sum = []
        ff = f
        for y in range(length):
            sum.append(data['open'][ff])
            ff = ff - 1
        data['dev_std'][f] = statistics.stdev(sum)

data['upper_bb'] = data['hullma'] + data['dev_std']
data['lower_bb'] = data['hullma'] - data['dev_std']

data['upperKC'] = data['hullma'] + data['rangema'] * multKC
data['lowerKC'] = data['hullma'] - data['rangema'] * multKC

# val = linreg(source - avg(avg(highest(high, lengthKC), lowest(low, lengthKC)), hma(close, lengthKC)), l

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to b

In [11]:
data

,index,date,symbol,open,high,low,close,Volume BTC,Volume USDT,predict,ap,esa,d,ci,wavetrend_1,wavetrend_2,wavetrend_delta,sqzwma1,sqzwma2,hullma,dev_std,TrueRange,rangewma1,rangewma2,rangema,upper_bb,lower_bb,upperKC,lowerKC
0,0,2017-08-17 04-AM,BTC/USDT,16199.91,16199.91,4261.32,4308.83,44.510000,1.909529e+05,False,8256.686667,8256.686667,0.000000,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0.0,0.0
1,1,2017-08-17 05-AM,BTC/USDT,4308.83,4328.69,4291.37,4315.32,23.230000,1.003048e+05,True,4311.793333,6086.995333,976.361100,-121.212121,-121.212121,NaN,NaN,0,0,0,0,37,0,0,0,0,0,0.0,0.0
2,2,2017-08-17 06-AM,BTC/USDT,4315.32,4345.45,4309.37,4324.35,7.230000,3.128231e+04,True,4326.390000,5379.243688,1007.110612,-69.694674,-94.226792,NaN,NaN,0,0,0,0,36,0,0,0,0,0,0.0,0.0
3,3,2017-08-17 07-AM,BTC/USDT,4324.35,4349.99,4287.41,4349.99,4.440000,1.924106e+04,True,4329.130000,5033.278015,907.297935,-51.739566,-78.695208,NaN,NaN,0,0,0,0,62,0,0,0,0,0,0.0,0.0
4,4,2017-08-17 08-AM,BTC/USDT,4349.99,4377.85,4333.32,4360.69,0.949900,4.139700e+03,True,4357.286667,4839.219284,785.187121,-40.918706,-67.861223,-90.498836,22.637613,0,0,0,0,44,0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35697,35697,2021-05-25 20:00:00,BTC/USDT,37292.69,37794.72,37250.01,37598.77,2513.621469,9.445717e+07,True,37547.833333,37712.280490,261.797694,-41.876396,-11.077599,-6.391099,-4.686500,37785,37660,38009,706,544,832,773,891,38715,37303,39345.5,36672.5
35698,35698,2021-05-25 21:00:00,BTC/USDT,37603.80,38110.00,37420.40,37937.79,2369.798232,8.940948e+07,True,37822.730000,37732.362219,230.628619,26.122165,-7.695802,-7.014090,-0.681713,37746,37678,37912,633,689,817,746,889,38545,37279,39245.5,36578.5
35699,35699,2021-05-25 22:00:00,BTC/USDT,37937.81,38753.98,37906.77,38405.00,3692.719609,1.412773e+08,True,38355.250000,37845.614543,281.357135,120.756480,3.981678,-5.697361,9.679039,37747,37747,37826,598,847,817,754,885,38424,37228,39153.5,36498.5
35700,35700,2021-05-25 23:00:00,BTC/USDT,38402.99,38467.22,37946.55,38324.72,2194.475099,8.392752e+07,False,38246.163333,37918.441596,289.787063,75.393690,10.473679,-1.079511,11.553190,37797,37875,37765,608,520,787,700,880,38373,37157,39085.0,36445.0


In [12]:
# Creating CONVERGENCE / DIVERGENCE OF THE MOBILE MEDIA (MACD)
fast_length = 12  # "Fast Length"
slow_length = 26  # "Slow Length"
signal_length = 9  # "Signal Smoothing"
src = "close"

data['fast_sma'] = data['close'].rolling(window=fast_length).mean()
data['fast_ema'] = data['close'].ewm(fast_length).mean()
data['fast_sma/ema'] = data['fast_sma'] / data['fast_ema']
data['slow_sma'] = data['close'].rolling(window=slow_length).mean()
data['slow_ema'] = data['close'].ewm(slow_length).mean()
data['slow_sma/ema'] = data['slow_sma'] / data['slow_ema']

data['macdsma'] = data['fast_sma'] - data['slow_sma']
data['macdema'] = data['fast_ema'] - data['slow_ema']
data['macd'] = data['fast_sma/ema'] - data['slow_sma/ema']

data['signalsma'] = data['macdsma'].rolling(window=signal_length).mean()
data['signalema'] = data['macdema'].rolling(window=signal_length).mean()
data['signalmacd'] = data['macd'].rolling(window=signal_length).mean()
data['signalmacdsmaema'] = data['signalsma'] / data['signalema']

data['hist'] = data['macd'] - data['signalmacd']
data['histsma'] = data['macdsma'] - data['signalsma']
data['histema'] = data['macdema'] - data['signalema']
data['histsmaema'] = data['macd'] - data['signalmacdsmaema']

In [13]:
# Clear first 40 records from the dataset
data = data.drop(range(0, 40), axis=0)

In [14]:
data

,index,date,symbol,open,high,low,close,Volume BTC,Volume USDT,predict,ap,esa,d,ci,wavetrend_1,wavetrend_2,wavetrend_delta,sqzwma1,sqzwma2,hullma,dev_std,TrueRange,rangewma1,rangewma2,rangema,upper_bb,lower_bb,upperKC,lowerKC,fast_sma,fast_ema,fast_sma/ema,slow_sma,slow_ema,slow_sma/ema,macdsma,macdema,macd,signalsma,signalema,signalmacd,signalmacdsmaema,hist,histsma,histema,histsmaema
40,40,2017-08-18 08-PM,BTC/USDT,4136.28,4136.28,3981.13,4021.11,18.940000,7.639196e+04,False,4046.173333,4185.575127,73.914622,-125.732266,-58.156311,-47.275167,-10.881143,4251,4211,4292,61,155,100,124,73,4353,4231,4401.5,4182.5,4234.505000,4246.903800,0.997081,4268.229615,4278.484295,0.997603,-33.724615,-31.580494,-0.000523,-18.903647,-16.254116,-0.000620,1.163007,0.000097,-14.820969,-15.326378,-1.163529
41,41,2017-08-18 09-PM,BTC/USDT,4021.11,4021.11,3938.77,3975.69,14.610000,5.804132e+04,False,3978.523333,4147.921115,91.278993,-123.721626,-64.238972,-54.056134,-10.182838,4227,4169,4289,82,82,99,119,76,4371,4207,4403.0,4175.0,4207.133333,4225.291849,0.995702,4254.785769,4264.379143,0.997750,-47.652436,-39.087294,-0.002048,-20.896510,-19.744070,-0.000281,1.058369,-0.001767,-26.755926,-19.343223,-1.060417
42,42,2017-08-18 10-PM,BTC/USDT,3975.69,4131.83,3938.77,4113.75,29.030000,1.173071e+05,True,4061.450000,4132.196281,87.545104,-53.874157,-63.279190,-59.226214,-4.052976,4201,4125,4283,103,193,109,135,79,4386,4180,4401.5,4164.5,4186.920000,4216.428012,0.993002,4246.651923,4257.428693,0.997469,-59.731923,-41.000681,-0.004467,-25.108362,-23.288216,-0.000449,1.078157,-0.004018,-34.623561,-17.712466,-1.082624
43,43,2017-08-18 11-PM,BTC/USDT,4113.75,4136.48,4085.88,4108.37,44.940000,1.847926e+05,False,4110.243333,4128.204252,74.891582,-15.988373,-58.907453,-61.145481,2.238028,4189,4113,4275,107,50,105,121,83,4382,4168,4399.5,4150.5,4167.591667,4207.862808,0.990430,4237.991154,4250.612773,0.997031,-70.399487,-42.749965,-0.006601,-31.077528,-26.881689,-0.001021,1.156085,-0.005580,-39.321959,-15.868276,-1.162686
44,44,2017-08-19 12-AM,BTC/USDT,4108.37,4178.31,4084.28,4163.49,41.220000,1.696942e+05,True,4142.026667,4130.717719,63.329719,11.904835,-52.371335,-59.699238,7.327903,4177,4104,4263,109,94,105,117,88,4372,4154,4395.0,4131.0,4153.172500,4204.353821,0.987827,4231.325385,4246.663279,0.996388,-78.152885,-42.309459,-0.008562,-38.343483,-30.223121,-0.001958,1.268680,-0.006603,-39.809402,-12.086338,-1.277242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35697,35697,2021-05-25 20:00:00,BTC/USDT,37292.69,37794.72,37250.01,37598.77,2513.621469,9.445717e+07,True,37547.833333,37712.280490,261.797694,-41.876396,-11.077599,-6.391099,-4.686500,37785,37660,38009,706,544,832,773,891,38715,37303,39345.5,36672.5,37549.390833,37718.558044,0.995515,38158.896923,37421.974631,1.019692,-609.506090,296.583413,-0.024177,-337.611261,351.856135,-0.018470,-0.959515,-0.005707,-271.894829,-55.272723,0.935338
35698,35698,2021-05-25 21:00:00,BTC/USDT,37603.80,38110.00,37420.40,37937.79,2369.798232,8.940948e+07,True,37822.730000,37732.362219,230.628619,26.122165,-7.695802,-7.014090,-0.681713,37746,37678,37912,633,689,817,746,889,38545,37279,39245.5,36578.5,37549.800833,37735.422040,0.995081,38094.731154,37441.078904,1.017458,-544.930321,294.343136,-0.022377,-387.694516,340.179067,-0.019475,-1.139678,-0.002902,-157.235805,-45.835930,1.117301
35699,35699,2021-05-25 22:00:00,BTC/USDT,37937.81,38753.98,37906.77,38405.00,3692.719609,1.412773e+08,True,38355.250000,37845.614543,281.357135,120.756480,3.981678,-5.697361,9.679039,37747,37747,37826,598,847,817,754,885,38424,37228,39153.5,36498.5,37658.280833,37786.928037,0.996595,38073.731154,37476.779685,1.015929,-415.450321,310.148352,-0.019333,-411.516795,333.007987,-0.019902,-1.235757,0.000569,-3.933526,-22.859635,1.216423
35700,35700,2021-05-25 23:00:00,BTC/USDT,38402.99,38467.22,37946.55,38324.7

In [15]:
data.columns.values

array(['index', 'date', 'symbol', 'open', 'high', 'low', 'close',
       'Volume BTC', 'Volume USDT', 'predict', 'ap', 'esa', 'd', 'ci',
       'wavetrend_1', 'wavetrend_2', 'wavetrend_delta', 'sqzwma1',
       'sqzwma2', 'hullma', 'dev_std', 'TrueRange', 'rangewma1',
       'rangewma2', 'rangema', 'upper_bb', 'lower_bb', 'upperKC',
       'lowerKC', 'fast_sma', 'fast_ema', 'fast_sma/ema', 'slow_sma',
       'slow_ema', 'slow_sma/ema', 'macdsma', 'macdema', 'macd',
       'signalsma', 'signalema', 'signalmacd', 'signalmacdsmaema', 'hist',
       'histsma', 'histema', 'histsmaema'], dtype=object)

In [16]:
x = data.drop('close', axis=1)
x = x.drop('predict', axis=1)
x = x.drop('date', axis=1)
x = x.drop('symbol', axis=1)
y = data['predict']
print (x)
print (y)

       index      open      high  ...     histsma    histema  histsmaema
40        40   4136.28   4136.28  ...  -14.820969 -15.326378   -1.163529
41        41   4021.11   4021.11  ...  -26.755926 -19.343223   -1.060417
42        42   3975.69   4131.83  ...  -34.623561 -17.712466   -1.082624
43        43   4113.75   4136.48  ...  -39.321959 -15.868276   -1.162686
44        44   4108.37   4178.31  ...  -39.809402 -12.086338   -1.277242
...      ...       ...       ...  ...         ...        ...         ...
35697  35697  37292.69  37794.72  ... -271.894829 -55.272723    0.935338
35698  35698  37603.80  38110.00  ... -157.235805 -45.835930    1.117301
35699  35699  37937.81  38753.98  ...   -3.933526 -22.859635    1.216423
35700  35700  38402.99  38467.22  ...  154.882507  -8.631559    1.251619
35701  35701  38324.72  38386.53  ...  248.033853   2.540364    1.242716

[35662 rows x 42 columns]
40       False
41       False
42        True
43       False
44        True
         ...  
35697  